In [1]:
import ee
 
# Trigger the authentication flow.
ee.Authenticate()
 
# Initialize the library.
ee.Initialize()

Enter verification code: 4/1AX4XfWh8LCOabXdJae8oS2x5Q9ev9e9w5h1BRioAvXleWquyNBJadTWQgBI

Successfully saved authorization token.


In [283]:
#import matplotlib.pyplot as plt
#import numpy as np
#from scipy.stats import norm, gamma, f, chi2
#import IPython.display as disp
#import json
import geojson
import geopandas as gpd
import pandas as pd
from glob import glob
from datetime import datetime

In [352]:
#Path to geojson polygon files containing geometries of agriculture fields
field_fc_pol = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/test_field_fc.geojson'
field_fc_p = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/test_field_p.geojson'
fields_01 = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/fields_01.geojson'

# Path to time series data per plot
ts_data = 'C://Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/ts_data/'

In [353]:
# Load geojson from file
with open(field_fc_pol) as f:
    data = geojson.load(f)

# Create GEE FeatureCollection from geojson file
geojsonFc = ee.FeatureCollection(data)
grid_geometry = geojsonFc.geometry().centroid().buffer(10000,1).bounds()

In [354]:
# Sentinel 1 GRD image collection PLOT
s1_plot = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(geojsonFc).filterDate(ee.Date('2018-03-01'), ee.Date('2018-09-30')) 

In [355]:
# Map reducer function over imagecollection to get mean for multipolygon geometries
s1_plot_rr = ee.FeatureCollection(s1_plot.map(lambda x: x.reduceRegions(collection=geojsonFc,reducer='mean', crs='EPSG:4326',scale=10)))

In [356]:
# Sentinel 1 GRD image collection GRID
s1_grid = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(geojsonFc).filterDate(ee.Date('2018-03-01'), ee.Date('2018-09-30'))

In [349]:
'''
mergeByDate is a function that merges images together that have the same date.

@ imgCol: [ee.ImageCollection] mandatory value that specifies the image collection to merge by dates with.

Returns ee.ImageCollection
'''
def mergeByDate(imgCol):
    #Convert the image collection to a list.
    imgList = imgCol.toList(imgCol.size())
    
    # Driver function for mapping the unique dates
    def uniqueDriver(image):
        return ee.Image(image).date().format("YYYY-MM-dd")
    
    uniqueDates = imgList.map(uniqueDriver).distinct()

    # Driver function for mapping the moasiacs
    def mosaicDriver(date):
        date = ee.Date(date)
        
        image = (imgCol
               .filterDate(date, date.advance(100, "day"))
               .mosaic())
        
        return image.set(
                        "system:time_start", date.millis(), 
                        "system:id", date.format("YYYY-MM-dd"))
    
    mosaicImgList = uniqueDates.map(mosaicDriver)
    
    return ee.ImageCollection(mosaicImgList)

s1_grid_test = mergeByDate(s1_grid)

In [357]:
# Load last corine land cover image 
clc = ee.Image.load('COPERNICUS/CORINE/V20/100m/2018').select('landcover')

# Mask areas where soil moisture measurements valid (farmland cat.:11-16)
clc_mask = clc.gte(211).And(clc.lte(231)) #binary map for updateMask

In [358]:
#NDVI
def maskS2clouds(image):
    qa = image.select('QA60')
    #Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    #Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    return image.updateMask(mask).divide(10000).copyProperties(image, ['system:time_start'])

def NDVI(image):
    ndvi = image.normalizedDifference(['nir','red']) #(first − second) / (first + second)
    return image.addBands(ndvi).rename(image.bandNames().add('NDVI')).copyProperties(image, ['system:time_start'])

def mask_ndvi(image):
    return image.lte(0.4).copyProperties(image, ['system:time_start'])

bandNamesOut_s2 = ['Aerosols','blue','green','red','red edge 1','red edge 2','red edge 3','nir','red edge 4','water vapor','cirrus','swir1','swir2','QA60']
bandNamesS2 = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B10','B11','B12','QA60']

s2_1c = ee.ImageCollection('COPERNICUS/S2').select(bandNamesS2,bandNamesOut_s2).filterDate(ee.Date('2018-03-01'), ee.Date('2018-09-30')).filterBounds(grid_geometry).map(maskS2clouds).map(NDVI)#.map(clip_aoi) 

ndvi_mask = s2_1c.select('NDVI').map(mask_ndvi)

# Add date to images properties
#ndvi_mask = ndvi_mask.map(lambda x: x.set('system:time_start' , ee.Date.parse('yyyyMMDD', ee.String(x.get('system:index')).slice(0,8))))

In [359]:
def mask_ni(image):
    d1 = image.date()
    s1 = image.mask(clc_mask)
    nmask = ndvi_mask.filterDate(d1,d1.advance(30,'days'))

    s1 = s1.mask(ndvi_mask.first())
    
    return s1.reduceRegions(collection=grid_geometry,reducer='mean', crs='EPSG:4326',scale=50).copyProperties(image, ['system:time_start'])

In [360]:
# Map reducer function over imagecollection to get mean for multipolygon geometries GRID
s1_grid_rr = ee.FeatureCollection(s1_grid.map(mask_ni))

In [361]:
# Get SAR backscatter time series signal for every plot and store it in a list PLOT
s1_plot_list = s1_plot_rr.toList(10000)
s1_plot_data = list()
for x in range(0,s1_plot_list.size().getInfo()):
    s1_plot_data.append(s1_plot_list.get(x).getInfo())

In [368]:
# Get orbit ASC or DES for s1 images PLOT
s1_orbit = s1_plot.getInfo()
orbit = list()
for x in s1_orbit['features']:
    orbit.append(x['properties']['orbitProperties_pass'])

In [370]:
# Create and save geojson file for every s1 grd timeseries a and b seperatly PLOT
for i,fc in enumerate(s1_plot_data):
    name = fc['properties']['system:index']
    orbit_name = '__' + orbit[i]
    with open(ts_data + '%s.geojson' %(name + '_plot_' + orbit_name), 'w') as file:
        geojson.dump(fc, file)

In [371]:
#Create geopandas geodataframe from all files PLOT
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in glob(ts_data + '*_plot*')],ignore_index=True))

In [380]:
#get dates as datetime.datetime for all filenames PLOt
dates = [datetime.strptime(x.split('\\')[1][17:32], '%Y%m%dT%H%M%S') for x in glob(ts_data + '*_plot*')]
sentinel = [x.split('\\')[1][2] for x in glob(ts_data + '*_plot*')]
orbit = [x.split('__')[1][1:4] for x in glob(ts_data + '*_plot*')]

In [381]:
# add dates to column by multiply dates by field numbers PLOT
gdf['date'] = np.array([[x] * len(gdf['id'].unique()) for x in dates]).flatten()

# add sentinel satellite description to column by multiply dates by field numbers
gdf['sentinel'] = np.array([[x] * len(gdf['id'].unique()) for x in sentinel]).flatten()

# add sentinel satellite description to column by multiply dates by field numbers
gdf['orbit'] = np.array([[x] * len(gdf['id'].unique()) for x in orbit]).flatten()

In [382]:
# save dataframe to disk PLOT
gdf.to_file('C://Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/test_field_ts2018_plot.geojson')

In [383]:
gdf

,id,VH,VV,angle,geometry,date,sentinel,orbit
0,0,-20.587427,-11.994290,32.970780,"POLYGON ((8.54584 49.79067, 8.54305 49.79019, ...",2018-03-04 05:42:17,A,DES
1,0,-22.329419,-11.926102,39.058007,"POLYGON ((8.54584 49.79067, 8.54305 49.79019, ...",2018-03-07 17:15:58,A,ASC
2,0,-18.911217,-10.769899,41.932689,"POLYGON ((8.54584 49.79067, 8.54305 49.79019, ...",2018-03-11 05:34:20,A,DES
3,0,-19.175302,-9.424283,39.054213,"POLYGON ((8.54584 49.79067, 8.54305 49.79019, ...",2018-03-19 17:15:58,A,ASC
4,0,-18.524833,-10.850781,41.935405,"POLYGON ((8.54584 49.79067, 8.54305 49.79019, ...",2018-03-23 05:34:20,A,DES
...,...,...,...,...,...,...,...,...
98,0,-19.399726,-10.308294,39.092154,"POLYGON ((8.54584 49.79067, 8.54305 49.79019, ...",2018-09-09 17:15:12,B,ASC
99,0,-19.471224,-11.280867,41.943629,"POLYGON ((8.54584 49.79067, 8.54305 49.79019, ...",2018-09-13 05:33:30,B,DES
100,0,-18.246122,-7.174835,32.971206,"POLYGON ((8.54584 49.79067, 8.54305 49.79019, ...",2018-09-18 05:41:36,B,DES
101,0,-22.730859,-11.560385,39.095526,"POLYGON ((8.54584 49.79067, 8.54305 49.79019, ...",2018-09-21 17:15:12,B,ASC


In [384]:
# Get SAR backscatter time series signal for every plot and store it in a list GRID
s1_grid_list = s1_grid_rr.toList(10000)
s1_grid_data = list()
for x in range(0,s1_grid_list.size().getInfo()):
    s1_grid_data.append(s1_grid_list.get(x).getInfo())

In [385]:
# Get orbit ASC or DES for s1 images GRID
s1_orbit = s1_grid.getInfo()
orbit = list()
for x in s1_orbit['features']:
    orbit.append(x['properties']['orbitProperties_pass'])

In [386]:
# Create and save geojson file for every s1 grd timeseries a and b seperatly GRID
for i,fc in enumerate(s1_grid_data):
    name = fc['properties']['system:index']
    orbit_name = '__' + orbit[i]
    with open(ts_data + '%s.geojson' %(name + '_grid_' + orbit_name), 'w') as file:
        geojson.dump(fc, file)

In [387]:
#Create geopandas geodataframe from all files GRID
gdf = gpd.GeoDataFrame(pd.concat([gpd.read_file(i) for i in glob(ts_data + '*_grid*')],ignore_index=True))

In [388]:
#get dates as datetime.datetime for all filenames GRID
dates = [datetime.strptime(x.split('\\')[1][17:32], '%Y%m%dT%H%M%S') for x in glob(ts_data + '*_grid*')]
sentinel = [x.split('\\')[1][2] for x in glob(ts_data + '*_grid*')]
orbit = [x.split('__')[1][1:4] for x in glob(ts_data + '*_grid*')]

In [389]:
# add dates to column by multiply dates by field numbers GRID
gdf['date'] = np.array([[x] * len(gdf['id'].unique()) for x in dates]).flatten()

# add sentinel satellite description to column by multiply dates by field numbers
gdf['sentinel'] = np.array([[x] * len(gdf['id'].unique()) for x in sentinel]).flatten()

# add sentinel satellite description to column by multiply dates by field numbers
gdf['orbit'] = np.array([[x] * len(gdf['id'].unique()) for x in orbit]).flatten()

In [390]:
# save dataframe to disk GRID
gdf.to_file('C://Users/USER/Desktop/Master_Irrigation/03_GIS/idm_test/test_field_ts2018_grid.geojson')

In [391]:
gdf

,id,VH,VV,angle,geometry,date,sentinel,orbit
0,0,-9.304641,-5.373621,15.930662,"POLYGON ((8.40546 49.69932, 8.68423 49.69932, ...",2018-03-04 05:42:17,A,DES
1,0,-9.726136,-5.667434,18.769203,"POLYGON ((8.40546 49.69932, 8.68423 49.69932, ...",2018-03-07 17:15:58,A,ASC
2,0,-5.851404,-3.362746,20.232563,"POLYGON ((8.40546 49.69932, 8.68423 49.69932, ...",2018-03-11 05:34:20,A,DES
3,0,-8.819704,-4.570282,18.767371,"POLYGON ((8.40546 49.69932, 8.68423 49.69932, ...",2018-03-19 17:15:58,A,ASC
4,0,-6.059996,-3.539010,20.233867,"POLYGON ((8.40546 49.69932, 8.68423 49.69932, ...",2018-03-23 05:34:20,A,DES
...,...,...,...,...,...,...,...,...
98,0,-5.008574,-2.994571,18.786112,"POLYGON ((8.40546 49.69932, 8.68423 49.69932, ...",2018-09-09 17:15:12,B,ASC
99,0,-9.780297,-5.903948,20.236771,"POLYGON ((8.40546 49.69932, 8.68423 49.69932, ...",2018-09-13 05:33:30,B,DES
100,0,-9.759325,-5.540011,15.930316,"POLYGON ((8.40546 49.69932, 8.68423 49.69932, ...",2018-09-18 05:41:36,B,DES
101,0,-5.030217,-2.888791,18.787740,"POLYGON ((8.40546 49.69932, 8.68423 49.69932, ...",2018-09-21 17:15:12,B,ASC


In [ ]:
#create poi for irrigated field at 2020-04-26
from shapely.geometry import Point
poi = Point(8.462525010108948, 49.84003060414562)

In [ ]:
# isolate field within point
field01 = gdf[gdf.contains(poi)]
field01

In [ ]:
import holoviews as hv
from holoviews import opts
hv.extension('bokeh')

In [3]:
# Import the Folium library.
import folium

# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

In [ ]:
import hvplot
import hvplot.pandas

In [ ]:
field01.hvplot(kind='scatter', x='date.day', y='VV', by='sentinel', xticks=30)

In [ ]:
# Create imagecollection by mapping reduceToImage function over FeatureCollection
s1_grd_rr_IC = ee.ImageCollection(s1_grd_rr.map(lambda x: ee.FeatureCollection(x).filter(ee.Filter.notNull(['VV'])).reduceToImage(properties=['VV'], reducer=ee.Reducer.first()).unmask(0).reproject('epsg:4326',None,10).clip(geojsonFc.geometry())))

In [ ]:
url = s1_grd_rr_IC.first().select('first').getThumbURL({'min': -20, 'max': 1})
disp.Image(url=url, width=800)

In [ ]:
ffa_db = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-04-20'), ee.Date('2020-04-30')) 
                       .first() 
                       .clip(aoi))
ffa_fl = ee.Image(ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') 
                       .filterBounds(aoi) 
                       .filterDate(ee.Date('2020-04-20'), ee.Date('2020-04-30')) 
                       .first() 
                       .clip(aoi))

In [ ]:
s1_grd_rr_IC.first().getInfo()

In [ ]:
#location = aoi.centroid().coordinates().getInfo()[::-1]

# Make an RGB color composite image (VV,VH,VV/VH).
#rgb = ee.Image.rgb(ffa_db.select('VV'),
#                   ffa_db.select('VH'),
#                   ffa_db.select('VV').divide(ffa_db.select('VH')))
test = s1_grd_rr_IC.first()
# Create the map object.
m = folium.Map(location=['49.983461596534646','8.471317291259766'], zoom_start=15)

# Add the S1 rgb composite to the map object.
m.add_ee_layer(test, {'min': [-20], 'max': [1]}, 'FFA')

# Add a layer control panel to the map.
m.add_child(folium.LayerControl())

# Display the map.
display(m)


In [ ]:
geoJSON = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              8.471317291259766,
              49.983461596534646
            ],
            [
              8.481616973876953,
              49.94867940605311
            ],
            [
              8.498611450195312,
              49.946249234682774
            ],
            [
              8.529338836669922,
              49.951882626433914
            ],
            [
              8.54710578918457,
              49.966239272192034
            ],
            [
              8.471317291259766,
              49.983461596534646
            ]
          ]
        ]
      }
    }
  ]
}
coords = geoJSON['features'][0]['geometry']['coordinates']
aoi_sub = ee.Geometry.Polygon(coords)


In [ ]:
hist = ffa_fl.select('VV').reduceRegion(
    ee.Reducer.fixedHistogram(0, 0.5, 500),aoi_sub).get('VV').getInfo()
mean = ffa_fl.select('VV').reduceRegion(
    ee.Reducer.mean(), aoi_sub).get('VV').getInfo()
variance = ffa_fl.select('VV').reduceRegion(
    ee.Reducer.variance(), aoi_sub).get('VV').getInfo()


In [ ]:
a = np.array(hist)
x = a[:, 0]                 # array of bucket edge positions
y = a[:, 1]/np.sum(a[:, 1]) # normalized array of bucket contents
plt.grid()
plt.plot(x, y, '.')
plt.show()


In [ ]:
alpha = 5
beta = mean/alpha
plt.grid()
plt.plot(x, y, '.', label='data')
plt.plot(x, gamma.pdf(x, alpha, 0, beta)/1000, '-r', label='gamma')
plt.legend()
plt.show()


In [ ]:
def X(n):
    return np.sum(np.cos(4*np.pi*(np.random.rand(n)-0.5)))/np.sqrt(n/2)

n= 10000
Xs = [X(n) for i in range(10000)]
y, x = np.histogram(Xs, 100, range=[-5,5])
plt.plot(x[:-1], y/1000, 'b.', label='simulated data')
plt.plot(x, norm.pdf(x), '-r', label='normal distribution')
plt.grid()
plt.legend()
plt.show()


In [ ]:
im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                .filterBounds(aoi)
                .filterDate(ee.Date('2020-08-01'),ee.Date('2020-08-31'))
                .filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))
                .filter(ee.Filter.eq('relativeOrbitNumber_start', 15))
                .sort('system:time_start'))


In [ ]:
import time
acq_times = im_coll.aggregate_array('system:time_start').getInfo()
[time.strftime('%x', time.gmtime(acq_time/1000)) for acq_time in acq_times]


In [ ]:
im_list = im_coll.toList(im_coll.size())
im1 = ee.Image(im_list.get(2)).select('VV').clip(aoi_sub)
im2 = ee.Image(im_list.get(5)).select('VV').clip(aoi_sub)

In [ ]:
ratio = im1.divide(im2)
url = ratio.getThumbURL({'min': 0, 'max': 10})
disp.Image(url=url, width=800)

In [ ]:
hist = ratio.reduceRegion(ee.Reducer.fixedHistogram(0, 5, 500), aoi_sub).get('VV').getInfo()
mean = ratio.reduceRegion(ee.Reducer.mean(), aoi_sub).get('VV').getInfo()
variance = ratio.reduceRegion(ee.Reducer.variance(), aoi_sub).get('VV').getInfo()


In [ ]:
im1 = ee.Image(im_list.get(2)).select('VV').clip(aoi)
im2 = ee.Image(im_list.get(5)).select('VV').clip(aoi)
ratio = im1.divide(im2)

location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(location=location, zoom_start=12)
mp.add_ee_layer(ratio,
                {'min': 0, 'max': 20, 'palette': ['black', 'white']}, 'Ratio')
mp.add_child(folium.LayerControl())

display(mp)


In [ ]:
# Decision threshold alpha/2:
dt = f.ppf(0.0005, 2*m, 2*m)

# LRT statistics.
q1 = im1.divide(im2)
q2 = im2.divide(im1)

# Change map with 0 = no change, 1 = decrease, 2 = increase in intensity.
c_map = im1.multiply(0).where(q2.lt(dt), 1)
c_map = c_map.where(q1.lt(dt), 2)

# Mask no-change pixels.
c_map = c_map.updateMask(c_map.gt(0))

# Display map with red for increase and blue for decrease in intensity.
location = aoi.centroid().coordinates().getInfo()[::-1]
mp = folium.Map(
    location=location, tiles='Stamen Toner',
    zoom_start=13)
folium.TileLayer('OpenStreetMap').add_to(mp)
mp.add_ee_layer(ratio,
                {'min': 0, 'max': 20, 'palette': ['black', 'white']}, 'Ratio')
mp.add_ee_layer(c_map,
                {'min': 0, 'max': 2, 'palette': ['black', 'blue', 'red']},
                'Change Map')
mp.add_child(folium.LayerControl())

display(mp)
